<a href="https://colab.research.google.com/github/Gorodecki/Student/blob/master/Plaid_to_google_Nekrasov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Сделать простой **cashflow** отчёт на основании данных полученных через Plaid из банка Bank of America

---
[пример решения](https://www.dropbox.com/s/lywnj7f1fchukkg/Screenshot%202020-05-24%2013.43.08.png?dl=0)


In [1]:
pip install plaid-python

  Created wheel for plaid-python: filename=plaid_python-4.0.0-cp36-none-any.whl size=30861 sha256=9865442e8a6dcb0196095e7e9e315ec00de765932c5a1f54d3a2efd4a08c8915
  Stored in directory: /root/.cache/pip/wheels/55/d4/d6/c8d3fbac7297d3dbddcbbc0cd31aeaff1144caa72aed923023
Successfully built plaid-python


In [0]:
import plaid
from plaid import Client
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

client = Client(client_id='5ecb985383a47600129f6cae',
                secret='7716367de7335f4d4c7a4b31de8d8f',
                public_key='2380fdba2a1117684cb356a8262e91',
                environment='sandbox')

In [3]:
SEARCH_QUERY = 'Bank of America' # bofa
response_bofa = client.Institutions.search(SEARCH_QUERY)
# search with products
response_bofa = client.Institutions.search(SEARCH_QUERY, products=['transactions'])
df = pd.DataFrame(response_bofa['institutions'])
df[['institution_id', 'name']]

,institution_id,name
0,ins_1,Bank of America
1,ins_100866,Bank of America - Charitable Gift Fund
2,ins_116859,First National Farmers Bank of America
3,ins_127288,Bank of America
4,ins_111305,Freedom Bank of America
5,ins_100865,Bank of America - Retirement
6,ins_127156,Bank of America Private Bank
7,ins_111266,First National Bank of America
8,ins_116966,Bank of America - Health Accounts
9,ins_119553,Bank of America - Money Network


In [4]:
INSTITUTION_ID = df['institution_id'][0]
print('INSTITUTION_ID = ', INSTITUTION_ID)
# Generate a public_token for a given institution ID
# and set of initial products
create_response = \
    client.Sandbox.public_token.create(INSTITUTION_ID,
        ['transactions'])
print(create_response)
# The generated public_token can now be
# exchanged for an access_token
exchange_response = \
    client.Item.public_token.exchange(
        create_response['public_token'])
print(exchange_response)

INSTITUTION_ID =  ins_1
{'public_token': 'public-sandbox-40ec3b3d-33d4-4102-b929-6de502d17863', 'request_id': 'nsfQIF9aHWCTAwz'}
{'access_token': 'access-sandbox-9da94992-9937-4d94-b7c2-d202805ad2cb', 'item_id': '9BGz3j3AybUvDo1j1qzNtGJpvq5DAjcRNkqPn', 'request_id': '9Y8AP6v7TzCq5JF'}


In [0]:
access_token = exchange_response['access_token']
all_transactions = []
response = client.Transactions.get(access_token,
                                   start_date='2018-01-01',
                                   end_date='2020-05-01')
transactions = response['transactions']

# Manipulate the count and offset parameters to paginate
# transactions and retrieve all available data
while len(transactions) < response['total_transactions']:
    response = client.Transactions.get(access_token,
                                       start_date='2018-01-01',
                                       end_date='2020-05-01',
                                       offset=len(transactions)
                                      )
    transactions.extend(response['transactions'])

time.sleep(3)

In [0]:
trans = pd.DataFrame(transactions)
trans.to_excel('plaid.xls', sheet_name='Sheet_1')

In [10]:
trans.head()

,account_id,account_owner,amount,authorized_date,category,category_id,date,iso_currency_code,location,name,payment_channel,payment_meta,pending,pending_transaction_id,transaction_code,transaction_id,transaction_type,unofficial_currency_code
0,dQGL49457yUAMx7D7vykuKWG6aolkXIPgBlBX,None,500.00,None,"[Travel, Airlines and Aviation Services]",22001000,2020-04-30,USD,"{'address': None, 'city': None, 'country': Non...",United Airlines,in store,"{'by_order_of': None, 'payee': None, 'payer': ...",False,None,None,ajoZxGx5NnTXkzRyR5nKhL7nJmvep5i7x6Zlg,special,None
1,B7rjeWe6yVfoRyrlr4GLtzyb4wEXjqu7ezXzL,None,6.33,None,"[Travel, Taxi]",22016000,2020-04-28,USD,"{'address': None, 'city': None, 'country': Non...",Uber,in store,"{'by_order_of': None, 'payee': None, 'payer': ...",False,None,None,eZbpNDN5e9UeXd717o9GCQPEqZWyp3iLlvrnx,special,None
2,dQGL49457yUAMx7D7vykuKWG6aolkXIPgBlBX,None,500.00,None,"[Food and Drink, Restaurants]",13005000,2020-04-25,USD,"{'address': None, 'city': None, 'country': Non...",Tectra Inc,in store,"{'by_order_of': None, 'payee': None, 'payer': ...",False,None,None,yBA7mRmeJPUa53zxzp89tgNQPp9VWdtyeBvll,place,None
3,dQGL49457yUAMx7D7vykuKWG6aolkXIPgBlBX,None,2078.50,None,[Payment],16000000,2020-04-24,USD,"{'address': None, 'city': None, 'country': Non...",AUTOMATIC PAYMENT - THANK,other,"{'by_order_of': None, 'payee': None, 'payer': ...",False,None,None,mErK1w1k3lCG7kKQKbrEiAp56KQGVNiLza3W6,special,None
4,dQGL49457yUAMx7D7vykuKWG6aolkXIPgBlBX,None,500.00,None,"[Food and Drink, Restaurants, Fast Food]",13005032,2020-04-24,USD,"{'address': None, 'city': None, 'country': Non...",KFC,in store,"{'by_order_of': None, 'payee': None, 'payer': ...",False,None,None,b78MdodJPbfRzB7x715GhjEDx9vlVKuVQevda,place,None


In [11]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   account_id                374 non-null    object 
 1   account_owner             0 non-null      object 
 2   amount                    374 non-null    float64
 3   authorized_date           0 non-null      object 
 4   category                  374 non-null    object 
 5   category_id               374 non-null    object 
 6   date                      374 non-null    object 
 7   iso_currency_code         374 non-null    object 
 8   location                  374 non-null    object 
 9   name                      374 non-null    object 
 10  payment_channel           374 non-null    object 
 11  payment_meta              374 non-null    object 
 12  pending                   374 non-null    bool   
 13  pending_transaction_id    0 non-null      object 
 14  transactio

In [12]:
data = trans[['date', 'category','amount']]
data

,date,category,amount
0,2020-04-30,"[Travel, Airlines and Aviation Services]",500.00
1,2020-04-28,"[Travel, Taxi]",6.33
2,2020-04-25,"[Food and Drink, Restaurants]",500.00
3,2020-04-24,[Payment],2078.50
4,2020-04-24,"[Food and Drink, Restaurants, Fast Food]",500.00
...,...,...,...
369,2018-06-08,"[Travel, Taxi]",6.33
370,2018-06-05,"[Food and Drink, Restaurants]",500.00
371,2018-06-04,[Payment],2078.50
372,2018-06-04,"[Food and Drink, Restaurants, Fast Food]",500.00


In [0]:
# меняем дату на DateTime
data['date'] =  pd.to_datetime( data['date'])
data.head()

,date,category,amount
0,2020-04-30,"[Travel, Airlines and Aviation Services]",500.00
1,2020-04-28,"[Travel, Taxi]",6.33
2,2020-04-25,"[Food and Drink, Restaurants]",500.00
3,2020-04-24,[Payment],2078.50
4,2020-04-24,"[Food and Drink, Restaurants, Fast Food]",500.00


In [13]:
# категорию делим на два уровня
data['category_one'] = 'None'
data['category_two'] = 'None'
for i, x in enumerate(data['category']):
  try:
    data['category_one'][i] = x[0]
  except:
    data['category_one'][i] = 'None'
  try:
    data['category_two'][i] = x[1]    
  except:
    data['category_two'][i] = 'None'
data.drop('category', axis=1, inplace=True)
data.head()

,date,amount,category_one,category_two
0,2020-04-30,500.00,Travel,Airlines and Aviation Services
1,2020-04-28,6.33,Travel,Taxi
2,2020-04-25,500.00,Food and Drink,Restaurants
3,2020-04-24,2078.50,Payment,None
4,2020-04-24,500.00,Food and Drink,Restaurants


**Positive** values when money **moves out** of the account

---


**Negative** values when money **moves in**

In [0]:
# разворачиваем цифры
data['amount'] = data['amount'] * -1
data['status'] = 'Expense'
data.loc[data['amount'] > 0,'status']  = 'Income'
data.info()

In [17]:
# приводим в порядок столбцы
new_data = pd.DataFrame(data, columns=['status', 'category_one', 'category_two', 'date', 'amount'])
print(new_data.info())
new_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   status        374 non-null    object 
 1   category_one  374 non-null    object 
 2   category_two  374 non-null    object 
 3   date          374 non-null    object 
 4   amount        374 non-null    float64
dtypes: float64(1), object(4)
memory usage: 14.7+ KB
None


,status,category_one,category_two,date,amount
0,Expense,Travel,Airlines and Aviation Services,2020-04-30,-500.00
1,Expense,Travel,Taxi,2020-04-28,-6.33
2,Expense,Food and Drink,Restaurants,2020-04-25,-500.00
3,Expense,Payment,None,2020-04-24,-2078.50
4,Expense,Food and Drink,Restaurants,2020-04-24,-500.00


In [0]:
# создаём обобщающую таблицу
netable = pd.pivot_table(new_data, values='amount', index=['status', 'category_one', 'category_two'],
                    columns=['date'], aggfunc=sum, fill_value=0, margins=True)
# сохраняем во второй лист
netable.to_excel('plaid_2.xls', sheet_name = 'Cashflow')

In [21]:
netable

date                                                   2018-06-04  ...        All
status  category_one   category_two                                ...           
Expense Food and Drink Restaurants                         -500.0  ...  -26431.79
        Payment        Credit Card                            0.0  ...    -575.00
                       None                               -2078.5  ...  -49884.00
        Recreation     Gyms and Fitness Centers               0.0  ...   -1805.50
        Shops          Sporting Goods                      -500.0  ...  -12000.00
        Transfer       Debit                                  0.0  ... -134550.00
                       Deposit                                0.0  ...  -23000.00
        Travel         Airlines and Aviation Services         0.0  ...  -12000.00
                       Taxi                                   0.0  ...    -276.12
Income  Transfer       Credit                                 0.0  ...      97.06
        Travel         Airlines and Aviation Services         0.0  ...   11500.00
All                                                       -3078.5  ... -248925.35

[12 rows x 235 columns]